In [1]:
import cv2
import mediapipe as mp
from datetime import datetime

# MediaPipe Pose 모델 초기화
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose()

# 비디오 파일 경로
video_path = 'C://Users//dokgo//Desktop//final//s111.mp4'

# 비디오 캡처 초기화
cap = cv2.VideoCapture(video_path)

# 키포인트 판별 threshold 값 설정
shoulder_hip_threshold = 0.4
ankle_thumb_threshold = 0.6

# 플랭크 자세 감지 여부
in_plank_pose = False

# 특정 시간 초기화
start_time = None

# 이전 프레임의 플랭크 자세 감지 여부
prev_in_plank_pose = False

# 이전 플랭크가 시작된 시간
prev_start_time = None

# 전체 경과 시간 (모든 시간을 계속 누적, 초 단위)
total_elapsed_seconds = 0

def is_plank(pose_landmarks):
    if pose_landmarks is None:
        return False

    # 양 어깨의 y좌표가 엉덩이의 y좌표보다 threshold 값 이상 위에 위치하는지 확인
    landmarks = pose_landmarks.landmark
    num_landmarks = len(landmarks)
    
    # Right side
    right_shoulder_y = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y if mp_pose.PoseLandmark.RIGHT_SHOULDER.value < num_landmarks else 0
    right_hip_y = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y if mp_pose.PoseLandmark.RIGHT_HIP.value < num_landmarks else 0
    right_thumb_y = landmarks[mp_pose.PoseLandmark.RIGHT_THUMB.value].y if mp_pose.PoseLandmark.RIGHT_THUMB.value < num_landmarks else 0

    # Left side
    left_shoulder_y = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y if mp_pose.PoseLandmark.LEFT_SHOULDER.value < num_landmarks else 0
    left_hip_y = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y if mp_pose.PoseLandmark.LEFT_HIP.value < num_landmarks else 0
    left_thumb_y = landmarks[mp_pose.PoseLandmark.LEFT_THUMB.value].y if mp_pose.PoseLandmark.LEFT_THUMB.value < num_landmarks else 0

    return (right_shoulder_y < right_hip_y and right_thumb_y > right_hip_y) or \
           (left_shoulder_y < left_hip_y and left_thumb_y > left_hip_y )

def seconds_elapsed(start_time, end_time):
    time_difference = end_time - start_time
    return time_difference.total_seconds()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # 현재 시간 가져오기
    current_time = datetime.now()

    # 플랭크 자세 감지 중이면
    if in_plank_pose:
        if start_time is None:
            # 플랭크 자세가 처음으로 감지된 시점을 기록
            start_time = current_time

        # 초로 변환
        elapsed_seconds = seconds_elapsed(start_time, current_time)

        # 전체 경과 시간 업데이트
        total_elapsed_seconds += elapsed_seconds

        # 프레임에 초 표시
        cv2.putText(frame, "Plank Pose Dectection", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
        cv2.putText(frame, f"Plank Time: {int((total_elapsed_seconds)/100)} seconds", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f"Total Kcal: {round(int((total_elapsed_seconds)/100)*0.3)} Kcal", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        

    # Mediapipe Pose 적용
    results = pose.process(frame_rgb)

    # 키포인트 감지된 경우
    if results.pose_landmarks is not None:
        # 랜드마크 좌표 시각화
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 0)))

        # 플랭크 자세 여부 판별
        in_plank_pose = is_plank(results.pose_landmarks)

        # 이전 프레임까지는 플랭크 자세가 감지되지 않았고,
        # 현재 프레임에서 플랭크 자세가 감지되면 시작 시간을 기록
        if in_plank_pose and not prev_in_plank_pose:
            prev_start_time = current_time

    # 이전 프레임에서는 플랭크 자세가 감지되었지만,
    # 현재 프레임에서는 감지되지 않으면 이전 시작 시간 초기화
    if prev_in_plank_pose and not in_plank_pose:
        start_time = None

    # 이전 프레임에서는 플랭크 자세가 감지되지 않았고,
    # 현재 프레임에서도 감지되지 않으면 이전 시작 시간 초기화
    if not prev_in_plank_pose and not in_plank_pose:
        start_time = None

    # 이전 프레임까지는 플랭크 자세가 감지되었지만,
    # 현재 프레임에서 감지되지 않으면 경과 시간을 멈추고 이전 시작 시간 초기화
    if prev_in_plank_pose and not in_plank_pose:
        elapsed_seconds = seconds_elapsed(prev_start_time, current_time)
        total_elapsed_seconds += elapsed_seconds
        cv2.putText(frame, "Plank Pose Dectection", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        start_time = None

    # 플랭크 자세 판정을 이전 값으로 업데이트
    prev_in_plank_pose = in_plank_pose

    # 비디오에 프레임 표시
    cv2.imshow("LWBM", frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 리소스 해제
cap.release()
cv2.destroyAllWindows()

# MediaPipe Pose 모델 해제
pose.close()
